In [ ]:
!pip install pydicom
!pip install torchcam
!pip install torchinfo
!pip install -U albumentations
!pip install iterative-stratification
!pip install ipywidgets
!pip install SciencePlots

In [ ]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchcam
import torchvision
from torchvision.models import shufflenet_v2_x1_0
from torchvision.transforms.functional import to_pil_image
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from torchcam.methods import GradCAM
from torchcam.utils import overlay_mask
from PIL import Image
import pydicom
import cv2
from sklearn.model_selection import train_test_split
from tqdm import tqdm

IMG_SIZE = 256
NUM_CLASSES = 6
BATCH_SIZE = 64
EPOCHS = 8
LEARNING_RATE = 2e-3
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class DicomDataset(Dataset):
    def __init__(self, df, image_dir, transform=None):
        self.df = df
        self.image_dir = image_dir
        self.transform = transform

    def correct_dcm(self, dcm):
        x = dcm.pixel_array + 1000
        x[x >= 4096] -= 4096
        dcm.PixelData = x.tobytes()
        dcm.RescaleIntercept = -1000

    def window_image(self, dcm, center, width):
        if dcm.BitsStored == 12 and dcm.PixelRepresentation == 0 and int(dcm.RescaleIntercept) > -100:
            self.correct_dcm(dcm)
        img = dcm.pixel_array * dcm.RescaleSlope + dcm.RescaleIntercept
        img = np.clip(img, center - width //2, center + width //2)
        return cv2.resize(img, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_AREA)

    def bsb_window(self, dcm):
        brain = (self.window_image(dcm, 40, 80)-0)/80
        subdural = (self.window_image(dcm, 80, 200)+20)/200
        soft = (self.window_image(dcm, 40, 380)+150)/380
        return np.stack([brain, subdural, soft], axis=-1).astype(np.float32)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.image_dir, row.name)
        dcm = pydicom.dcmread(img_path)
        img = self.bsb_window(dcm)
        img = np.transpose(img, (2,0,1)) 
        label = torch.tensor(row.values, dtype=torch.float32)
        return {'image': torch.tensor(img), 'labels': label}

df = pd.read_csv('../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_train.csv')
df['sub_type'] = df['ID'].str.split('_', expand=True)[2]
df['image'] = 'ID_' + df['ID'].str.split('_', expand=True)[1] + '.dcm'
df = df.pivot_table(index='image', columns='sub_type', values='Label', aggfunc='first').fillna(0)

sample_df = df.sample(frac=0.0075, random_state=42)
train_df, val_df = train_test_split(sample_df, test_size=0.15, random_state=42)
image_dir = '../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_train/'

train_ds = DicomDataset(train_df, image_dir)
val_ds = DicomDataset(val_df, image_dir)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

In [ ]:
model = shufflenet_v2_x1_0(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
model.to(DEVICE)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        inputs = batch['image'].to(DEVICE)
        labels = batch['labels'].to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Loss: {running_loss / len(train_loader):.4f}")

In [ ]:
class_label = ['epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural', 'any']
model.eval()
cam_extractor = GradCAM(model, target_layer='conv5')

for idx in range(30):
    sample = val_ds[idx]
    input_tensor = sample['image'].unsqueeze(0).to(DEVICE)
    label = sample['labels'].numpy()

    output = model(input_tensor)
    preds = torch.sigmoid(output).cpu().detach().numpy()[0]
    top_class = int(np.argmax(preds))

    activation_map = cam_extractor(top_class, output)[0]
    img_np = input_tensor.squeeze(0).cpu().numpy()
    img_np = (img_np - img_np.min()) / (img_np.max() - img_np.min() + 1e-8)
    img_uint8 = (img_np * 255).astype(np.uint8)
    img_uint8 = np.transpose(img_uint8, (1, 2, 0))
    img_pil = Image.fromarray(img_uint8)
    cam_pil = to_pil_image(activation_map, mode='F')
    result = overlay_mask(img_pil, cam_pil, alpha=0.5)

    print(f"\nSample no {idx+1}")
    print("Actual:   ", label)
    print("Predicted:", preds)
    print("Top class:", class_label[top_class])
    plt.imshow(result)
    plt.axis('off')
    plt.show()